<a href="https://colab.research.google.com/github/sundar911/topic_modeling/blob/main/LDA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis==2.1.2
!pip install gensim==3.8.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gensim
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser
from pprint import pprint
import re  # For preprocessing
from collections import defaultdict  # For word frequency
from gensim.models import Phrases
from gensim.models.phrases import Phrases, Phraser
from wordcloud import WordCloud
from gensim import corpora

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
df_covid = pd.read_csv('/content/drive/MyDrive/Tourism | NITT Sundar/covid_final.csv')

In [ ]:
df_covid.shape

(18644, 22)

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(df_covid.cleaned_review.astype(str)))
bigram_model = Phrases(data_words)
trigram_model = Phrases(bigram_model[data_words], min_count=1)
data_words = list(trigram_model[bigram_model[data_words]])

In [ ]:
# Create Dictionary and filtering out least (appearing in less than 1.5% of the reviews (assuming appears only once in each review)) and most frequent words (appearing in more than 70% of the reviews)
id2word = corpora.Dictionary(data_words)
id2word.filter_extremes(no_above=0.70, no_below=280)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text, allow_update=True) for text in texts]

print(corpus[:1][0][:30])

In [ ]:
mallet_9 = gensim.models.wrappers.LdaMallet(mallet_path,
                                            corpus=corpus,
                                            num_topics=9,
                                            id2word=id2word,
                                            random_seed=100)

model_9 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(mallet_9)
LDAvis_prepared = pyLDAvis.gensim.prepare(model_9, corpus, id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(LDAvis_prepared)

INFO - 04:04:23: serializing temporary corpus to /tmp/d9ca24_corpus.txt
INFO - 04:04:25: converting temporary corpus to MALLET format with /content/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/d9ca24_corpus.txt --output /tmp/d9ca24_corpus.mallet
INFO - 04:04:28: training MALLET LDA with /content/mallet-2.0.8/bin/mallet train-topics --input /tmp/d9ca24_corpus.mallet --num-topics 9  --alpha 50 --optimize-interval 0 --num-threads 4 --output-state /tmp/d9ca24_state.mallet.gz --output-doc-topics /tmp/d9ca24_doctopics.txt --output-topic-keys /tmp/d9ca24_topickeys.txt --num-iterations 1000 --inferencer-filename /tmp/d9ca24_inferencer.mallet --doc-topics-threshold 0.0  --random-seed 100
INFO - 04:06:23: loading assigned topics from /tmp/d9ca24_state.mallet.gz
INFO - 04:06:26: using serial LDA version on this node
INFO - 04:06:29: NumExpr defaulting to 2 threads.
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.p



1.   complaints #2
2.   property, scenery, amenities #5
3.   overall experience #6
4.   compliments to hotel team #8
5.   resort location/leisure travel #7
6.   hospitality #4
7.   quality of stay and safety measures taken by hotel #3
8.   staff behaviour and compliance with covid norms #1
9.   service quality and norms followed during meals #0






